In [1]:
from transformers import pipeline
import torch
import torch.nn.functional as F

In [2]:
classifier = pipeline("sentiment-analysis")
res = classifier("It's weird having to write a test statement")

res

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'NEGATIVE', 'score': 0.9991441965103149}]

In [3]:
torch.cuda.is_available()

True

In [4]:
from datasets import load_dataset

In [5]:
ds = load_dataset("imdb")
sm_train_ds = ds["train"].shuffle(seed=42).select([i for i in list(range(3000))])
sm_test_ds = ds["test"].shuffle(seed=42).select([i for i in list(range(300))])

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [7]:

tokenized_train = sm_train_ds.map(lambda training_data: tokenizer(training_data["text"], truncation=True))
tokenized_test = sm_test_ds.map(lambda training_data: tokenizer(training_data["text"], truncation=True))

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [8]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
import numpy as np
import evaluate

def make_metrics(eval_pred):
    try:
        logits, labels = eval_pred

        if logits is None or labels is None:
            raise ValueError("Predictions or labels are None. Check your model output and dataset.")

        predictions = np.argmax(logits, axis=1)

        load_accuracy = evaluate.load("accuracy")
        load_f1 = evaluate.load("f1")
        accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
        f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]

        return {"accuracy": accuracy, "f1": f1}
    
    except Exception as e:
        print(f"Error in compute_metrics: {e}")
        return {"accuracy": 0.0, "f1": 0.0}
    
# Test
logits = np.array([[2.5, 0.3], [0.2, 1.8], [0.6, 2.1]])
labels = np.array([0, 1, 1])
eval_pred = (logits, labels)

metrics_result = make_metrics(eval_pred)
print(f"Metrics result: {metrics_result}")


Logits (predictions): [[2.5 0.3]
 [0.2 1.8]
 [0.6 2.1]]
Labels: [0 1 1]
Metrics result: {'accuracy': 1.0, 'f1': 1.0}


In [11]:
from huggingface_hub import notebook_login
notebook_login()

In [13]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
repo = "Falcontreras/Tiny_Sentiment_Tunning"

training_args = TrainingArguments(
    output_dir = repo,
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    gradient_accumulation_steps=2,
    num_train_epochs = 2,
    weight_decay = 0.01,
    save_strategy = "epoch",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_test,
    processing_class = tokenizer,
    data_collator = data_collator,
    compute_metrics = make_metrics,
)

In [15]:
trainer.train()

  0%|          | 0/188 [00:00<?, ?it/s]

{'train_runtime': 7983.9042, 'train_samples_per_second': 0.752, 'train_steps_per_second': 0.024, 'train_loss': 0.7169969436970163, 'epoch': 2.0}


TrainOutput(global_step=188, training_loss=0.7169969436970163, metrics={'train_runtime': 7983.9042, 'train_samples_per_second': 0.752, 'train_steps_per_second': 0.024, 'total_flos': 782725021021056.0, 'train_loss': 0.7169969436970163, 'epoch': 2.0})

In [17]:
trainer.evaluate()

  0%|          | 0/19 [00:00<?, ?it/s]

Logits (predictions): [[-1.6293926   1.7726997 ]
 [-1.1313399   1.2758397 ]
 [ 1.8944573  -1.4557424 ]
 [-1.3110665   1.4755783 ]
 [ 1.1447257  -0.88943374]
 [-1.6273776   1.7475519 ]
 [-1.5640956   1.6001297 ]
 [ 1.8975282  -1.55583   ]
 [ 1.2811943  -0.88485193]
 [-1.6648817   1.7681725 ]
 [-1.6094394   1.7663562 ]
 [ 2.025593   -1.5489964 ]
 [ 0.84684384 -0.6577347 ]
 [ 1.7370695  -1.3016437 ]
 [-1.2548065   1.2693152 ]
 [-1.4687142   1.5796375 ]
 [ 1.5047259  -1.0991029 ]
 [ 0.07051201  0.2427731 ]
 [-1.5749729   1.7036742 ]
 [-1.5331188   1.7154322 ]
 [ 1.7671981  -1.343828  ]
 [ 0.49133915 -0.2101544 ]
 [ 1.24735    -0.94446653]
 [-1.7186016   1.7815206 ]
 [ 1.8298724  -1.5154271 ]
 [-1.6680976   1.7243953 ]
 [ 1.8006867  -1.4741155 ]
 [-1.2293732   1.3555249 ]
 [-1.683152    1.8024626 ]
 [-1.7485244   1.8053206 ]
 [-1.1763717   0.98514134]
 [ 1.2347395  -1.1087644 ]
 [-0.9073483   0.92064005]
 [ 1.5867493  -1.2808142 ]
 [-1.0612026   1.1954702 ]
 [-1.381774    1.4805082 ]
 [ 1.2

{'eval_loss': 0.29375752806663513,
 'eval_model_preparation_time': 0.004,
 'eval_accuracy': 0.8766666666666667,
 'eval_f1': 0.8794788273615635,
 'eval_runtime': 22.7712,
 'eval_samples_per_second': 13.175,
 'eval_steps_per_second': 0.834,
 'epoch': 2.0}

In [16]:
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Falcontreras/Tiny_Sentiment_Tunning/commit/6b6c6557caf56911a93860861f51887a56f503a0', commit_message='End of training', commit_description='', oid='6b6c6557caf56911a93860861f51887a56f503a0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Falcontreras/Tiny_Sentiment_Tunning', endpoint='https://huggingface.co', repo_type='model', repo_id='Falcontreras/Tiny_Sentiment_Tunning'), pr_revision=None, pr_num=None)

In [4]:
from transformers import pipeline
 
sentiment_model = pipeline(model="Falcontreras/Tiny_Sentiment_Tunning")
sentiment_model(["What a shitty company omg", "This was a great game, I appreciate the graphics", "Uh, I wouldn't try it out"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_0', 'score': 0.9007973074913025},
 {'label': 'LABEL_1', 'score': 0.9501630067825317},
 {'label': 'LABEL_0', 'score': 0.6908736228942871}]